In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, explode
from pyspark.sql.types import StringType, ArrayType

In [3]:
# Iniciando sessão no Spark
spark = SparkSession.builder.appName('Cluster').getOrCreate()

23/06/05 09:25:34 WARN Utils: Your hostname, daniel-VJFE43F11X-XXXXXX resolves to a loopback address: 127.0.1.1; using 192.168.0.157 instead (on interface wlo1)
23/06/05 09:25:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/05 09:25:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Importando dados
dados_cep = spark.read.csv('../data/external/postcode_ranges.xlsx - Sheet1.csv', header=True, sep=',')

In [5]:
# Vendo os dados
dados_cep.show()

+-----+--------------------+--------------------+--------------------+--------------------+------------+--------------------+------------+--------------------+
|state|            locality|      postcode_range|           situation|          range_type|        page|        fetched_from|fetched_date|    url_fetched_from|
+-----+--------------------+--------------------+--------------------+--------------------+------------+--------------------+------------+--------------------+
|   AC|          Acrelândia|69945-000 a 69949...|Não codificada po...|  Total do município|1 a 23 de 23|Busca Faixa CEP o...|  2022-02-13|https://buscacepi...|
|   AC|        Assis Brasil|69935-000 a 69939...|Não codificada po...|  Total do município|1 a 23 de 23|Busca Faixa CEP o...|  2022-02-13|https://buscacepi...|
|   AC|           Brasiléia|69932-000 a 69933...|Não codificada po...|  Total do município|1 a 23 de 23|Busca Faixa CEP o...|  2022-02-13|https://buscacepi...|
|   AC|              Bujari|69926-000 a 

Desde conjunto, irei precisar apenas da localidade e do postcode. Para um resultado
satisfatório, irei precisar remover o suffix do postcode, possibilitando assim o
Join com outras tabelas que possuem apenas o prefixo da cidade.


In [6]:
# Selecionando colunas
dados_cep = dados_cep.select('locality', 'postcode_range')

In [7]:
# Criando funções para separar os CEPs
func_ceps = lambda cep: cep.split(' a ')
func_remove_sufix = lambda cep: [cep[0].split('-')[0], cep[1].split('-')[0]]
func_busca_todos_ceps = lambda cep: [cep for cep in range(int(cep[0]), int(cep[1])+1)]

# Criando UDFs
ceps_udf = udf(func_ceps, StringType())
prefix_cep = udf(func_remove_sufix, StringType())
todos_os_ceps = udf(func_busca_todos_ceps, ArrayType(StringType()))

In [8]:
# Aplicando as funções
dados_cep = dados_cep.withColumn('ceps', ceps_udf('postcode_range'))
dados_cep = dados_cep.withColumn('cep_prefixes', prefix_cep('ceps'))
dados_cep = dados_cep.withColumn('todos_cep_prefix', todos_os_ceps('cep_prefixes'))

In [9]:
# Criando uma linha com cada CEP
dados_to_save = dados_cep.select(dados_cep.locality, explode(dados_cep.todos_cep_prefix).alias('cep_prefix'))

In [10]:
dados_to_save.show()

+---------------+----------+
|       locality|cep_prefix|
+---------------+----------+
|     Acrelândia|     69945|
|     Acrelândia|     69946|
|     Acrelândia|     69947|
|     Acrelândia|     69948|
|     Acrelândia|     69949|
|   Assis Brasil|     69935|
|   Assis Brasil|     69936|
|   Assis Brasil|     69937|
|   Assis Brasil|     69938|
|   Assis Brasil|     69939|
|      Brasiléia|     69932|
|      Brasiléia|     69933|
|         Bujari|     69926|
|       Capixaba|     69931|
|Cruzeiro do Sul|     69980|
|Cruzeiro do Sul|     69981|
| Epitaciolândia|     69934|
|          Feijó|     69960|
|          Feijó|     69961|
|          Feijó|     69962|
+---------------+----------+
only showing top 20 rows



In [12]:
# Salvando os dados no formato parquet
dados_to_save.write.parquet('../data/external/cep_prefixes_processed.parquet', mode='overwrite')